In [6]:
import pandas as pd

# Load the files
matches_df = pd.read_csv('matches.csv')
deliveries_df = pd.read_csv('deliveries.csv')

# Merge the datasets based on match_id
merged_df = pd.merge(matches_df, deliveries_df, left_on='id', right_on='match_id')

# Select relevant columns
selected_columns = ['match_id', 'venue', 'inning', 'batting_team', 'bowling_team', 'batsman_runs', 'batsman']
merged_df = merged_df[selected_columns]

# Calculate total runs scored by each batsman in a match
total_runs_df = merged_df.groupby(['match_id', 'batsman']).agg({
    'venue': 'first',
    'inning': 'first',
    'batting_team': 'first',
    'bowling_team': 'first',
    'batsman_runs': 'sum'
}).reset_index()
total_runs_df.rename(columns={'batsman_runs': 'total_runs'}, inplace=True)

# Calculate number of 4s and 6s
fours_sixes_df = merged_df.groupby(['match_id', 'batsman']).agg({
    'batsman_runs': lambda x: (x == 4).sum()
}).reset_index()
fours_sixes_df.rename(columns={'batsman_runs': 'no_of_4s'}, inplace=True)

sixes_df = merged_df.groupby(['match_id', 'batsman']).agg({
    'batsman_runs': lambda x: (x == 6).sum()
}).reset_index()
sixes_df.rename(columns={'batsman_runs': 'no_of_6s'}, inplace=True)

# Merge total runs, fours, and sixes dataframes
batsmen_runs_df = pd.merge(total_runs_df, fours_sixes_df, on=['match_id', 'batsman'], how='left')
batsmen_runs_df = pd.merge(batsmen_runs_df, sixes_df, on=['match_id', 'batsman'], how='left')

# Save the resulting dataframe as a separate CSV file
batsmen_runs_df.to_csv('batsmen_runs.csv', index=False)


In [5]:
import pandas as pd

# Load the files
matches_df = pd.read_csv('matches.csv')
deliveries_df = pd.read_csv('deliveries.csv')

# Merge the datasets based on match_id
merged_df = pd.merge(matches_df, deliveries_df, left_on='id', right_on='match_id')

# Select relevant columns
selected_columns = ['match_id', 'venue', 'inning', 'batting_team', 'bowling_team', 'player_dismissed', 'bowler']
merged_df = merged_df[selected_columns]

# Calculate total wickets taken by each bowler in a match
total_wickets_df = merged_df.groupby(['match_id', 'venue', 'batting_team', 'bowling_team', 'inning', 'bowler']).apply(lambda x: (x['player_dismissed'].notnull()).sum()).reset_index(name='wickets')

# Save the resulting dataframe as a separate CSV file
total_wickets_df.to_csv('bowlers_wickets.csv', index=False)
